In [9]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tqdm.auto import tqdm 

In [10]:
import ast
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tqdm.auto import tqdm
import random

# --- КОНФИГУРАЦИЯ ---
model_name = "iiiorg/piiranha-v1-detect-personal-information"
data_file = "/kaggle/input/dataset3/more_pi_dataset.csv" 
tokenized_dataset_path = "./tokenized_log_dataset"

# --- ЗАГРУЗКА ---
print("--- Шаг 1: Загрузка токенизатора и датасета ---")
tokenizer = AutoTokenizer.from_pretrained(model_name)
full_dataset = load_dataset("csv", data_files=data_file, split="train")
print("Датасет загружен. Всего записей:", len(full_dataset))


# --- ФУНКЦИЯ ПРЕОБРАЗОВАНИЯ ---
def prepare_dataset(examples):
    examples["tokens"] = [ast.literal_eval(tok_list) for tok_list in examples["mbert_tokens"]]
    examples["ner_labels"] = [ast.literal_eval(label_list) for label_list in examples["mbert_token_classes"]]
    return examples

print("\n--- Шаг 2: Преобразование строковых колонок в списки ---")
columns_to_remove = [col for col in full_dataset.column_names if col not in ['mbert_tokens', 'mbert_token_classes']]
prepared_dataset = full_dataset.map(
    prepare_dataset,
    batched=True,
    num_proc=2,
    remove_columns=columns_to_remove,
    desc="Парсинг токенов и меток"
)
print("Преобразование завершено.")


# --- ИЗМЕНЕНИЕ: НОВЫЙ ШАГ БАЛАНСИРОВКИ ---
print("\n--- Шаг 2.5: Балансировка датасета (Undersampling) ---")

# Функция для проверки, есть ли в примере хоть одна PII метка
def has_pii(example):
    # set(example['ner_labels']) - {'O'} вернет True, если есть что-то кроме 'O'
    return bool(set(example['ner_labels']) - {'O'})

# Разделяем датасет на две части
pii_dataset = prepared_dataset.filter(has_pii, num_proc=2)
no_pii_dataset = prepared_dataset.filter(lambda x: not has_pii(x), num_proc=2)

print(f"Найдено {len(pii_dataset)} записей с PII.")
print(f"Найдено {len(no_pii_dataset)} записей без PII (только 'O').")

# Определяем, сколько "пустых" записей мы хотим оставить.
# Сделаем их количество равным количеству записей с PII (соотношение 1:1)
num_pii_samples = len(pii_dataset)

# Перемешиваем "пустые" записи и выбираем нужное количество
# seed=42 делает выбор случайным, но воспроизводимым
if len(no_pii_dataset) > num_pii_samples:
    print(f"Уменьшаю количество записей без PII с {len(no_pii_dataset)} до {num_pii_samples}.")
    no_pii_dataset_balanced = no_pii_dataset.shuffle(seed=42).select(range(num_pii_samples))
else:
    print("Количество записей без PII уже меньше или равно количеству записей с PII. Undersampling не требуется.")
    no_pii_dataset_balanced = no_pii_dataset

# Объединяем два датасета обратно в один, теперь сбалансированный
balanced_dataset = concatenate_datasets([pii_dataset, no_pii_dataset_balanced])
# Перемешиваем финальный датасет, чтобы PII и не-PII примеры шли вперемешку
balanced_dataset = balanced_dataset.shuffle(seed=42)

print(f"Размер нового сбалансированного датасета: {len(balanced_dataset)} записей.")


# --- Получение меток из модели ---
model_config = AutoModelForTokenClassification.from_pretrained(model_name).config
id2label = {int(k): v for k, v in model_config.id2label.items()}
label2id = {v: k for k, v in id2label.items()}
print("\nСловарь меток (label2id) из модели:", label2id)


# --- ФУНКЦИЯ ТОКЕНИЗАЦИИ (остается без изменений) ---
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=512,
    )
    all_labels = []
    for i, ner_tags in enumerate(examples["ner_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                tag = ner_tags[word_idx]
                label_ids.append(label2id.get(tag, label2id["O"]))
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)
    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

# --- ОБРАБОТКА И СОХРАНЕНИЕ ---
print("\n--- Шаг 3: Начинаю токенизацию сбалансированных данных ---")
# Применяем токенизацию к нашему новому `balanced_dataset`
tokenized_datasets = balanced_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=2,
    remove_columns=balanced_dataset.column_names,
    desc="Токенизация логов"
)
print("Токенизация завершена.")


print(f"\n--- Шаг 4: Сохраняю обработанный датасет в '{tokenized_dataset_path}' ---")
tokenized_datasets.save_to_disk(tokenized_dataset_path)
print("Подготовка данных успешно завершена!")

--- Шаг 1: Загрузка токенизатора и датасета ---


Generating train split: 0 examples [00:00, ? examples/s]

Датасет загружен. Всего записей: 100

--- Шаг 2: Преобразование строковых колонок в списки ---


Парсинг токенов и меток (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Преобразование завершено.

--- Шаг 2.5: Балансировка датасета (Undersampling) ---


...

Найдено 100 записей с PII.
Найдено 0 записей без PII (только 'O').
Количество записей без PII уже меньше или равно количеству записей с PII. Undersampling не требуется.
Размер нового сбалансированного датасета: 100 записей.

Словарь меток (label2id) из модели: {'I-ACCOUNTNUM': 0, 'I-BUILDINGNUM': 1, 'I-CITY': 2, 'I-CREDITCARDNUMBER': 3, 'I-DATEOFBIRTH': 4, 'I-DRIVERLICENSENUM': 5, 'I-EMAIL': 6, 'I-GIVENNAME': 7, 'I-IDCARDNUM': 8, 'I-PASSWORD': 9, 'I-SOCIALNUM': 10, 'I-STREET': 11, 'I-SURNAME': 12, 'I-TAXNUM': 13, 'I-TELEPHONENUM': 14, 'I-USERNAME': 15, 'I-ZIPCODE': 16, 'O': 17}

--- Шаг 3: Начинаю токенизацию сбалансированных данных ---


Токенизация логов (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Токенизация завершена.

--- Шаг 4: Сохраняю обработанный датасет в './tokenized_log_dataset' ---


Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Подготовка данных успешно завершена!


In [11]:
!pip install seqeval
!pip install evaluate

In [12]:
# import transformers
# print(transformers.__version__)

In [13]:
# !pip install --upgrade --force-reinstall "numpy<2" "datasets" "transformers" "torch" "evaluate" "seqeval" "scikit-learn" "tqdm"

In [14]:
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
    pipeline  # <-- ИЗМЕНЕНИЕ 1: Импортируем pipeline для удобного предсказания
)
import evaluate
import numpy as np
import pandas as pd # <-- ИЗМЕНЕНИЕ 2: Импортируем pandas для красивого вывода
from tqdm.auto import tqdm

# --- КОНФИГУРАЦИЯ ---
model_name = "iiiorg/piiranha-v1-detect-personal-information"
tokenized_dataset_path = "./tokenized_log_dataset"
output_dir = "piiranha-finetuned-logs"
final_model_path = f"{output_dir}-final"

# --- ЗАГРУЗКА ---
print("Загружаю подготовленный датасет...")
tokenized_datasets_from_disk = load_from_disk(tokenized_dataset_path)
dataset_dict = tokenized_datasets_from_disk.train_test_split(test_size=0.1, seed=42) # Добавим seed для воспроизводимости

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# --- МЕТРИКИ (ОБНОВЛЕННАЯ ВЕРСИЯ) ---
seqeval = evaluate.load("seqeval")
label_list = list(model.config.id2label.values())

# ИЗМЕНЕНИЕ 3: Расширяем функцию метрик для детального отчета
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    # Получаем полный отчет от seqeval
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    
    # Собираем метрики по каждому классу PII
    per_class_results = {}
    for key, value in results.items():
        if isinstance(value, dict) and 'f1-score' in value:
             per_class_results[f"{key}_precision"] = value['precision']
             per_class_results[f"{key}_recall"] = value['recall']
             per_class_results[f"{key}_f1"] = value['f1-score']

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
        **per_class_results # Добавляем метрики по классам
    }

# --- НАСТРОЙКИ ОБУЧЕНИЯ ---
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3, # Возможно, стоит увеличить до 5-10, раз модель не учится
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=10,
    report_to="none",
)

# --- ОБУЧЕНИЕ ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Начинаю дообучение модели...")
trainer.train()

# --- СОХРАНЕНИЕ ---
trainer.save_model(final_model_path)
print(f"\nОбучение завершено. Модель сохранена в '{final_model_path}'")

# --- ИЗМЕНЕНИЕ 4: ФУНКЦИЯ ДЛЯ ПРОСМОТРА ПРЕДСКАЗАНИЙ ---
def show_predictions(model_path, num_examples=3):
    """Загружает модель и показывает ее предсказания на нескольких примерах из тестового сета."""
    print("\n--- Анализ предсказаний модели ---")
    
    # Загружаем лучшую сохраненную модель
    ner_pipe = pipeline("token-classification", model=model_path, aggregation_strategy="simple")
    
    test_set = dataset_dict["test"]
    
    for i in range(num_examples):
        if i >= len(test_set):
            break
        
        example = test_set[i]
        tokens = tokenizer.convert_ids_to_tokens(example['input_ids'])
        true_labels = [label_list[l] if l != -100 else "PAD" for l in example['labels']]
        
        # Склеиваем токены обратно в текст для pipeline
        text = tokenizer.decode(example['input_ids'], skip_special_tokens=True)
        
        print(f"\n--- Пример #{i+1} ---")
        print(f"Текст: {text}")
        
        predictions = ner_pipe(text)
        
        # Создаем словарь предсказаний для удобства
        pred_map = {}
        for pred in predictions:
            # Pipeline может склеивать B- и I- токены. Нам нужно найти начальный токен.
            start_token_index = tokenizer(pred['word'], add_special_tokens=False).input_ids[0]
            for tok_idx in tokenizer(pred['word'], add_special_tokens=False).input_ids:
                 pred_map[tok_idx] = pred['entity_group']

        # Создаем DataFrame для наглядного сравнения
        results_df = pd.DataFrame({
            "Токен": tokens,
            "Настоящая метка": true_labels
        })
        
        # Добавляем предсказания, если они есть
        predicted_labels = []
        for j, token_id in enumerate(example['input_ids']):
            if true_labels[j] == "PAD": # Пропускаем паддинг
                continue
            
            # Находим предсказание для этого токена
            # ВАЖНО: этот метод не идеален, т.к. pipeline агрегирует токены,
            # но для визуальной проверки он хорошо подходит.
            # Более точный способ требует ручного прогона модели и сопоставления.
            # Сейчас для простоты оставим так.
            
            # Более простой и надежный способ: прогоним токенизированный ввод через модель
        
    # Более точный способ без pipeline
    print("\n--- Точный анализ предсказаний (без pipeline) ---")
    model.eval()
    for i in range(num_examples):
        example = test_set[i]
        input_ids = torch.tensor(example['input_ids']).unsqueeze(0).to(model.device)
        
        with torch.no_grad():
            logits = model(input_ids).logits
        
        predictions = torch.argmax(logits, dim=2).squeeze().tolist()
        
        tokens = tokenizer.convert_ids_to_tokens(example['input_ids'])
        true_labels = [label_list[l] if l != -100 else "PAD" for l in example['labels']]
        pred_labels = [label_list[p] for p in predictions]
        
        df = pd.DataFrame({
            'Токен': tokens, 
            'Реальная метка': true_labels, 
            'Предсказание модели': pred_labels
        })
        
        # Фильтруем PAD и спец.токены для чистоты вывода
        df_filtered = df[(df['Реальная метка'] != 'PAD') & (~df['Токен'].isin(['[CLS]', '[SEP]', '[PAD]']))]
        
        print(f"\n--- Пример #{i+1} ---")
        
        # Выводим только строки, где есть расхождения или где есть PII
        mismatches = df_filtered[df_filtered['Реальная метка'] != df_filtered['Предсказание модели']]
        real_pii = df_filtered[df_filtered['Реальная метка'] != 'O']
        
        display_df = pd.concat([real_pii, mismatches]).drop_duplicates().sort_index()

        if display_df.empty:
            print("Расхождений не найдено, и в примере нет PII.")
        else:
            print(display_df.to_string())

# Запускаем нашу новую функцию
import torch # Нужно для точного анализа
show_predictions(final_model_path, num_examples=5)

Загружаю подготовленный датасет...


/tmp/ipykernel_37/1118594090.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Начинаю дообучение модели...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.356286,0.040000,0.047619,0.043478,0.922942
2,0.414200,0.181793,0.068966,0.095238,0.080000,0.940455
3,0.414200,0.177600,0.080000,0.095238,0.086957,0.937828


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


RuntimeError: [enforce fail at inline_container.cc:626] . unexpected pos 1129750976 vs 1129750864

In [ ]:
!nvidia-smi